In [4]:
import numpy as np
import matplotlib.pyplot as plt
import math
from enum import Enum

# 第五章　換気設備の計算方法

## はじめに


本計算は第五章「換気設備」に基づく。

### 機械換気設備の消費電力量


#### 定義

機械換気設備の消費電力量$ E_{E,V,d,t} $は式(1)に基づく。  
$$
\begin{equation}
    \displaystyle E_{E,V,d,t} = E_{E,VG,d,t} + E_{E,VL,d,t} \tag{1}  
\end{equation}
$$

<br />
ここで、  
$ E_{E,V,d,t} $：機械換気設備の消費電力量(kWh/h)  
$ E_{E,VG,d,t} $：全般換気設備の消費電力量(kWh/h)  
$ E_{E,VL,d,t} $：局所換気設備の消費電力量(kWh/h)  
である。

In [2]:
def get_E_E_V(E_E_VG, E_E_VL):
    return E_E_VG + E_E_VL

#### 例題

以下の条件で計算する。  
$ E_{E,VG,d,t} $は0.3kWh/h  
$ E_{E,VL,d,t} $は0.1kWh/h


In [3]:
get_E_E_V(0.3, 0.1)

0.4

### 全般換気設備の消費電力量

#### 定義

全般換気設備の消費電力量($ E_{E,VG,d,t} $)は式(2)により表される。  
$$
\begin{equation}
    E_{E,VG,d,t} = f_{SFP} \times V_R \times 10^{-3} \tag{2}
\end{equation}
$$
<br />
ここで、  
$ f_{SFP} $：全般換気設備の比消費電力(W/(m3/h))  
$ V_R $：全般換気設備の参照機械換気量(m3/h)  
である。

In [4]:
def get_E_E_VG(f_SFP, V_R):
    return f_SFP * V_R * 10**(-3)

#### 例題

以下の条件で計算する。  
$ f_{SFP} $は0.3  
$ V_R $は3.5

In [5]:
get_E_E_VG(0.3, 3.5)

0.0010500000000000002

#### 定義

全般換気設備の参照換気量($V_R$)は、式(3)により表される。  
$$
\begin{equation}
    V_R = A_A \times H_R \times N \times a \div e \tag{3}
\end{equation}
$$

<br />
ここで、  
$ A_A $：床面積の合計(m2)  
$ H_R $：参照天井高さ(m)(=2.4m)  
$ N $：換気回数(1/h)  
$ a $：全般換気設備の換気量の余裕率(=1.1)  
$ e $：有効換気量率  
である。  



In [6]:
def get_V_R(A_A, N, e):
    H_R = 2.4 # 参照天井高さ(m)
    a = 1.1 # 全般換気設備の換気量の余裕率
    return A_A * H_R * N * a / e

#### 例題
以下の条件で計算する。  
$A_A$は10  
$N$は5  
$e$は1

In [7]:
get_V_R(10,5,1)

132.0

#### 定義

比消費電力$ f_{SFP} $は、全般換気設備、ダクト及び電動機の種別に応じて定まる付録Aに規定する値を用いるか、又は式(4)により算出し、1/100未満の端数を切り上げた小数第二位までの値とする。  
$$
\begin{equation}
    f_{SFP} = P \div V_d \tag{4}
\end{equation}
$$

<br />
ここで、  
$ P $：全般換気設備の消費電力(W)  
$ V_d $：全般換気設備の設計風量(m3/h)  
である。

In [5]:
# 付録A 全般換気設備の比消費電力
class Type(Enum):
    duct1_EXCH    =1 # 0.70　ダクト式第一種換気設備（熱交換型換気設備）
    duct1         =2 # 0.50　ダクト式第一種換気設備
    duct2or3      =3 # 0.40　ダクト式第二種換気設備又はダクト式第三種換気設備
    wall1_EXCH    =4 # 0.70　壁付け式第一種換気設備（熱交換型換気設備）
    wall1         =5 # 0.40　壁付け式第一種換気設備
    wall2         =6 # 0.30　壁付け式第二種換気設備
    wall3         =7 # 0.30　壁付け式第三種換気設備
    
class Saving(Enum):
    duct1_over75mm_DC    = 1 # 0.455　ダクト式第一種換気設備、内径75mm以上のダクトのみ使用、直流
    duct1_over75mm_AC    = 2 # 0.700　ダクト式第一種換気設備、内径75mm以上のダクトのみ使用、交流、又は直流と交流の併用
    duct1_other          = 3 # 1.000　ダクト式第一種換気設備、上記以外
    duct2or3_over75mm_DC = 4 # 0.360　ダクト式第二種又は第三種換気設備、内径75mm以上のダクトのみ使用、直流
    duct2or3_over75mm_AC = 5 # 0.600　ダクト式第二種又は第三種換気設備、内径75mm以上のダクトのみ使用、交流、又は直流と交流の併用
    duct2or3_other       = 6 # 1.000　ダクト式第二種又は第三種換気設備、上記以外

    
def get_f_SFP(P = 0, V_d = 0, get_Type = 0 , get_Saving = 0):
    # 式(4)により算出する場合
    if get_Type == 0 and get_Saving == 0:
        return math.ceil(P / V_d * 100) / 100
    
    # 付録Aに規定する値を用いる場合
    if P == 0 and V_d == 0:
        if get_Type == Type(1).value: # ダクト式第一種換気設備（熱交換型換気設備）
            Value1 = 0.70
        elif get_Type == Type(2).value: # ダクト式第一種換気設備
            Value1 = 0.50
        elif get_Type == Type(3).value: # ダクト式第二種換気設備又はダクト式第三種換気設備
            Value1 = 0.40
        elif get_Type == Type(4).value: # 壁付け式第一種換気設備（熱交換型換気設備）
            Value1 = 0.70
        elif get_Type == Type(5).value: # 壁付け式第一種換気設備
            Value1 = 0.40
        elif get_Type == Type(6).value: # 壁付け式第二種換気設備
            Value1 = 0.30
        elif get_Type == Type(7).value: # 壁付け式第三種換気設備
            Value1 = 0.30

        if get_Type == Type(1).value or get_Type == Type(2).value:
            if get_Saving == Saving(1).value:
                Value2 = 0.455
            elif get_Saving == Saving(2).value:
                Value2 = 0.700
            elif get_Saving == Saving(3).value:
                Value2 = 1.000
            else:
                Value2 = 1.000

        if get_Type == Type(3).value:
            if get_Saving == Saving(4).value:
                Value2 = 0.360
            elif get_Saving == Saving(5).value:
                Value2 = 0.600
            elif get_Saving == Saving(6).value:
                Value2 = 1.000
            else:
                Value2 = 1.000

        return Value1 * Value2
    
    else:
        return 1




#### 例題
以下の条件で計算する。  
$P$は50  
$V_d$は270

In [59]:
get_f_SFP(50,270,0,0)

0.19

#### 例題
以下の条件で計算する。  
ダクト式第二種換気設備で、内径75mm以上のダクトのみを使用し、直流の電動機

In [58]:
get_f_SFP(0,0,3,4)

0.144

### 局所換気設備の消費電力量

#### 定義

局所換気設備の消費電力量($E_{E,VL,d,t}$)は、式(5)により表される。  
$ E_{E,VL,d,t} = \left\{
\begin{array}{ll}
E_{E,VL,p,d,t} \mid _{p=1} \times \frac{2 - n_p}{2 - 1} + E_{E,VL,p,d,t} \mid _{p=2} \times \frac{n_p - 1}{2 - 1} & (1 \leq n_p < 2 )\\
E_{E,VL,p,d,t} \mid _{p=2} \times \frac{3 - n_p}{3 - 2} + E_{E,VL,p,d,t} \mid _{p=3} \times \frac{n_p - 2}{3 - 2} & (2 \leq n_p < 3)\\ \tag{5}
E_{E,VL,p,d,t} \mid _{p=3} \times \frac{4 - n_p}{4 - 3} + E_{E,VL,p,d,t} \mid _{p=3} \times \frac{n_p - 3}{4 - 3} & (3 \leq n_p < 4)
\end{array}
\right.$  

<br />
ここで、  
$ E_{E,VL,d,t} $：日付dの時刻tにおける1時間当たりの局所換気設備の消費電力量(kWh/h)  
$ E_{E,VL,p,d,t} $：日付dの時刻tにおける1時間当たりの居住人数がp人における局所換気設備の消費電力量(kWh/h)  
$ n_p $：仮想居住人数  
である。

In [12]:
def get_E_E_VL(E_E_VL_p, E_E_VL_p_plus1, n_p):
    if 1 <= n_p < 2:
        return E_E_VL_p * (2 - n_p) / (2 - 1) + E_E_VL_p_plus1  * (n_p - 1) / (2 - 1)
    if n_p == 2 <= n_p < 3:
        return E_E_VL_p * (3 - n_p) / (3 - 2) + E_E_VL_p_plus1  * (n_p - 2) / (3 - 2)
    if n_p == 3 <= n_p < 4:
        return E_E_VL_p * (4 - n_p) / (4 - 3) + E_E_VL_p_plus1  * (n_p - 3) / (4 - 3)
   

#### 説明
例題として以下の条件で計算する。  
$n_p=1.4$  
$E_{E,VL,p}=0.00054$  
$E_{E,VL,p+1}=0.00108$

In [13]:
get_E_E_VL(0.00054, 0.00108, 1.4)

0.0007559999999999999

# 計算方法（統合）

### $f_{SFP}$の設定に課題あり。

In [14]:
def result(N, n_p, A_A, e, P, V_d, E_E_VL_p, E_E_VL_p_plus1):
    
    # 局所換気設備の消費電力量
    E_E_VL = get_E_E_VL(E_E_VL_p, E_E_VL_p_plus1, n_p)
    print('局所換気の消費電力量' + str(E_E_VL))
    
    # 全般換気設備の消費電力量
    V_R = get_V_R(A_A, N, e)
    f_SFP = get_f_SFP_calc(P, V_d)
    E_E_VG = get_E_E_VG(f_SFP, V_R)
    print('全般換気設備の消費電力量' + str(E_E_VG))
    
    # 機械換気設備の消費電力量
    E_E_V = get_E_E_V(E_E_VG, E_E_VL)
    print('機械換気設備の消費電力量' + str(E_E_V))
    

In [15]:
result( 0.5, 1.4, 120, 1, 50, 270, 0.15, 0.5)

局所換気の消費電力量0.29
全般換気設備の消費電力量0.030096
機械換気設備の消費電力量0.320096
